In [68]:
import pydub
from skimage import io,color
import numpy as np
import pandas as pd
image_path = "/home/billbrod/Pictures/WFB Brussels.jpg"

In [76]:
img = io.imread(image_path)
#We want to use the L*a*b color space instead of RGB because it approximates human perceptual space
lab = color.rgb2lab(img)

In [77]:
def color_df(arr,col,color_name):
    df=pd.DataFrame(arr[:,:,col])
    df['x'] = df.index
    df = pd.melt(df,id_vars='x',var_name='y',value_name=color_name)
    return df

In [80]:
L,a,b = [color_df(i,j,k) for i,j,k in zip([lab,lab,lab],[0,1,2],['L','a','b'])]
df = pd.merge(L,pd.merge(a,b))

Based on http://stackoverflow.com/a/29906003, $L \in [0,100], a,b \in [-128,127]$. We need to know our min and max in order to find our mappable range.

Our color space is three dimensional ($L,a,b$) and for sound to be three dimensional, we're going to use octave ($L$), tone ($a$) and volume ($b$). Make those re-mappable, but we'll see how that works.